**Objetivo general**: a través de este laboratorio pretendemos establecer buenas prácticas en el proceso de análisis de datos y modelos predictivos.

**Objetivo particular**: implementar un esquema completo desde el análisis de los datos hasta la construcción del modelo de regresión que permita predecir costos de viviendas partiendo de diferentes atributos de las mismas.

---



**Parte 1**: Análisis y visualización de datos
  *   Tipos de variables.
  *   Identificación de valores faltantes.
  *   Identificación de valores atípicos (outliers).
  *   Correlación de variables.

---

**Parte 2**: Modelo
  * Ingeniería de características.
  * Modelo predictivo.
    * Entrenamiento, validación y test.

### Dataset
La base de datos contiene información sobre atributos utilizados para tasar propiedades residenciales en la ciudad de Ames (Iowa).

- **Período**: 2006 - 2010
- **Cantidad de variables**: 82
- **Fuente**: Oficina de Tasación de inmuebles de Ames (Iowa)
- **Información adicional**
   -  http://jse.amstat.org/v19n3/decock.pdf
   -  [codebook](https://drive.google.com/file/d/1pDkSyI8UHtLEFdjpAVNmsVqB8N6a5Pqv/view?usp=sharing)

## Parte 2: Entrenar modelos de regresión

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

!pip install plotly==5.3.1                  
import plotly.express as px                 # Librería que permite realizar visualizaciones interactivas 
                                            # Algunas funciones usadas se encuentran en versiones más actuales: desde la  4.12.0
                                            # Si tiene una versión anterior, instalar: !pip install plotly==5.3.1
# Consultar versión de plotly instalada
import plotly
print(plotly.__version__)

In [ ]:
# Leemos el dataset
path = 'https://drive.google.com/uc?export=download&id=1UVZnskEk-GZbTo4uW2usL7ze92XVDS8_'
df = pd.read_csv(path)

# Información básica del dataset
#df.info()

*Notar que existen atributos en los que se pueden encontrar algunos (o muchos) **valores faltantes**. Dados por la diferencia entre la cantidad de entradas y la cantidad de valores ```non-null```*

In [ ]:
missings = df.isna().sum()
missings[missings > 0].sort_values()

In [ ]:
# Porcentaje de valores faltantes

missings = df.isna().sum()      # Chequeamos los valores nulos y los contamos.
missings[missings > 0].sort_values()  # Filtramos solo aquellos que tienen valores nulos

missing_percentage = (missings[missings > 0] / len(df) *100).sort_values()
missing_percentage

In [ ]:
attributes_to_drop = missing_percentage[missing_percentage > 16.0].index.to_list()
print('Los atributes a eliminar:', attributes_to_drop)

In [ ]:
df.drop(attributes_to_drop, axis=1, inplace=True)

Similarmente, podemos optar por eliminar aquellas pocas filas que pierden atributos. EL siguiente código funciona para eliminar la fila donde falta el valor para ```Electrical``` pero puede adaptarse para eliminar otros también.

In [ ]:
# Buscamos el (los) indices donde faltan pocos atributos.
attributes_least_missing = ['Electrical', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtUnfSF', 'BsmtHalfBath', 'BsmtFullBath']

idx_to_drop = []
for attribute in attributes_least_missing:
  idx_to_drop.extend(df[df[attribute].isna()].index)
print('Eliminamos las filas:', idx_to_drop)

# Usamos el índice para eliminarlo de la tabla
df.drop(idx_to_drop, inplace=True)

In [ ]:
df

In [ ]:
# Eliminar alguna fila hace que quede  discontinuo el índice del dataframe (Index)
# pero se puede resetear:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
missings = missing_percentage[missing_percentage <= 16.0].index.to_list()

for a in attributes_least_missing:
  missings.remove(a)
missings

Si observamos el mapa de correlación, podemos ver que GarageYrBlt tiene alta correlación con YearBlt. Por lo tanto podemos eliminar esta variable.

Se pueden considerar distintas alternativas. En muchos casos suele ser factible reemplazar los valores faltantes con los valores más frecuentes (moda), media o mediana.

Otra opción puede ser intentar inferir estos valores a través de algún modelo de clasificación (usarndo las otras variables para entrenar el modelo)

Por simplicidad vamos a eliminar estas variables. Apoyamos esta decisión por algunas observaciones como: El número de entradas es considerablemente bajo comparado con los otros casos. Las variables que tienen el mismo prefijo tienen la misma cantidad de valores nulos (probablemente se deba a que esos faltantes son en las mismas entradas (chequear)).


In [ ]:
df.drop(missings, axis=1, inplace=True)

In [ ]:
df.info()

Ahora tenemos un dataset limpio para pensar en el el modelo que queremos utilzar.

Dado que vamos a usar un modelo de regresión tendremos que chequear que se cumplan ciertas condiciones.

Además, tenemos que poder representar las variables categóricas de manera numérica para finalmente entrenar y usar nuestro modelo predictivo.

### Ingeniería de características

Por un lado tenemos las características ordinales (listas para usar)

Por el otro, las variables categóricas (incluidas los códigos numéricos que no presentan orden). Todas estas necesitan una representación numérica adecuada.

Notar que tampoco necesitamos usar Order ni PID para nuestro entrenamiento

In [ ]:
# Dividimos las features segun son ordinales o categoricas
# Usar el codeBook para asegurarnos la correcta división

#Notar incluso que hay una variable de tipo Object pero en la descripción 
# se indica que es ordinal (HeatingQC):
# Ex	Excellent        -> 5
# Gd	Good             -> 4
# TA	Average/Typical  -> 3
# Fa	Fair             -> 2
# Po	Poo              -> 1

df['HeatingQC'] = df['HeatingQC'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po': 1})

In [ ]:
categorical = ['MSSubClass',
                'MSZoning',
                'Street',
                'LotShape',
                'LandContour',
                'Utilities',
                'LotConfig',
                'LandSlope',
                'Neighborhood',
                'Condition1',
                'Condition2',
                'BldgType',
                'HouseStyle',
                'RoofStyle',
                'RoofMatl',
                'Exterior1st',
                'Exterior2nd',
                'ExterQual',
                'ExterCond',
                'Foundation',
                'Heating',
                'CentralAir',
                'Electrical',
                'KitchenQual',
                'Functional',
                'PavedDrive',
                'SaleType',
                'SaleCondition']
df[categorical]

#### Tratamiento de variables categóricas como vectores *one-hot*

Por ejemplo:

In [ ]:
# Ejemplo de codificación tipo "One-hot"
# Nos permite resolver el problema de no tener orden entre los distintos valores categóricos.
pd.get_dummies(df['PavedDrive'], prefix='PavedDrive')

In [ ]:
# Necesitamos este tipo de codificación para todas las variables categóricas
# Podemos directamente aplicar este tipo de codificación sobre el dataFrame completo
# e indicar cuáles son las columnas categóricas:

df_encoded = pd.get_dummies(data=df, columns=categorical)


In [ ]:
# Separamos la variable objetivo del resto (es decir, dividimos el dataset en "X e Y" )
sale_price = df_encoded[['SalePrice']]                   # Y
df_encoded.drop('SalePrice', axis=1, inplace= True)      # X

# Prescindimos de Order y PID (si bien no es necesaria esta información, la guardo por las dudads)
order_pid = df_encoded[['Order', 'PID']]

In [ ]:
df_encoded

### Ejemplo 1 (completar)

En este ejemplo vamos a construir un modelo de regresión lineal simple
y para ellos usaremos SOLO UNA variable (elijan aquella que crean más adecuada):

* Crear un modelo de regresión lineal (usando scikit-learn).
* Usar como entrenamiento un solo atributo (tomar el que crean más adecuado).
* Para entrenar, dividir los datos en entrenamiento y evaluación.
(Como tenemos solo un atributo, podemos graficar facilmente)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics # por si quieren usar otras métricas.

### Elegir una feature
### Crear X (dado el dataframe y la feature elegida)

X = # >> completar <<<
y = df['SalePrice']

# Generar el split train/test usando la función train_test_split:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# >> completar <<

# Instanciar la clase que provee la regresión lineal
# ajustar el modelo
# >> completar <<

# Generar la predicción
y_pred = # >> completar <<

# Usar alguna métrica como MSE u otra para imprimir el error
#print('MSE:', # >> completar << ...

# Plotear los puntos de entrenamiento
# >> completar <<

# Plotear los puntos de test
# >> completar <<

# Plotear la recta de regresión
# >> completar <<

plt.show()




#### Repetimos con más de una features

In [ ]:
features = # >> completar <<
X = # >> completar <<
y = df['SalePrice']

# Generar el split train/test usando la función train_test_split
# Instanciar el modelo y ajustar
# Calcular el error y mostrar el resultado.
# >> completar <<



**Extra**: generar el mismo procedimiento pero iterando en cantidad de features que agregamos, guardar los errores y ver gráficamente (features y error).

La idea es ver si a medida que agregamos features el error disminuye.

### Ejemplo 2 (completar)

De manera similar al ejemplo visto en el teórico, validar modelos polinómicos de distintos grados y generar una visualización que permita comparar los errores en el conjunto de entrenamiento y evaluación.

Recordar generar las particiones (splits) necesarias para entrenar, validar y evaluar.

In [ ]:

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split



X = # completar
y = df['SalePrice']

# Split train+val y test
# train_test_split(...) # completar

train_errors = []
val_errors = []

degrees = [1, 2, 3, 4, 5, 6, 7]
for M in degrees:
    ### train ###
    # Usar PolynomialFeatures y LinearRegression
    # para construir el modelo polinómico,
    # apoyarse en make_pipeline para crear y ajustar el modelo.
    
    # completar
    
    
    # Generar las predicciones
    y_train_pred = # completar
    y_val_pred = # completar
    
    ### Evaluar ###
    # Usar alguna métrica (mean_squared_error, por ejemplo)
    # para guardar los errores de train y de test.
    train_error = # copmletar
    val_error = # completar
    train_errors.append(train_error)
    val_errors.append(val_error)

In [ ]:
# graficar
plt.show()

In [ ]:
# Una vez comparados los modelos
# entrenar con todo el conjunto de entrenamiento y evaluar.
M = # completar
